# Graphs - Playground

A place to develop a custom graph module

In [43]:
import numpy as np

In [129]:
class Graph:    
    def __init__(self,nv=1,directed=False):
        '''Create empty graph'''
        self.adj = {}
        self.nv = nv
        self.directed = directed # Plug for the future
        for i in range(self.nv):
            self.adj[i] = []
            
    def __str__(self):
        return f'Graph of {len(self.adj)} edges:\n'+'\n'.join([str(key)+':'+str(self.adj[key]) for key in self.adj])
    
    def add_edge(self,i,j):
        if i not in self.adj: 
            self.adj[i] = [j]
        else:                 
            self.adj[i].append(j)
        if not self.directed:
            if j not in self.adj: 
                self.adj[j] = [i]
            else:                 
                self.adj[j].append(i)
        else:
            if j not in self.adj: 
                self.adj[j] = []
                
    def add_edges(self,ts):
        '''Accepts a list of tuples'''
        for (i,j) in ts:
            self.add_edge(i,j)
                
    def adj_matrix(self):
        out = np.zeros((len(self.adj),len(self.adj)),dtype=int)
        for i in self.adj:
            for j in self.adj[i]:
                out[i,j] = 1
        return out
    
    def dfs(self,v=None,visited=None,path=[],verbose=False):
        '''Depth-first exploration, purposeless for now'''
        if v is None: v=next(iter(self.adj.keys())) # First vertex (whatever)
        if visited is None: visited = [0]*len(self.adj)
        visited[v] = 1
        path += [v]
        if verbose: print(path)
        for i in self.adj[v]:
            if visited[i]==0:
                self.dfs(i,visited,path)
                
    def bfs(self,v=None,target=None,verbose=False):
        if v is None: v = next(iter(self.adj.keys())) # One whatever vertex, put in a list
        q = [v] # Queue
        dl = [0]
        visited = [0]*len(self.adj)
        while len(q)>0:
            v = q.pop(-1)
            d = dl.pop(-1) # Degree of separation (min distance)
            if v==target:
                return d
            if verbose: print(v,q)
            visited[v] =  1
            for i in self.adj[v]:
                if visited[i]==0 and (i not in q):
                    q.append(i)
                    dl.append(d+1)                    
        
            
g = Graph()
g.add_edges([(0,2),(0,1),(1,2),(2,3),(4,5)])
print(g)
#g.adj_matrix()
print('dfs:')
g.dfs(verbose=True)
print('bfs:')
g.bfs(verbose=True)
print(g.bfs(0,3))
print(g.bfs(3,0))
print(g.bfs(2,5))

Graph of 6 edges:
0:[2, 1]
2:[0, 1, 3]
1:[0, 2]
3:[2]
4:[5]
5:[4]
dfs:
[0]
bfs:
0 []
1 [2]
2 []
3 []
2
2
None


In [128]:
g = Graph(directed=True)
g.add_edges([(0,2),(0,1),(1,2),(2,3),(4,5)])
print(g)
#g.adj_matrix()
print('dfs:')
g.dfs(verbose=True)
print('bfs:')
g.bfs(verbose=True)
print(g.bfs(0,3))
print(g.bfs(3,0))
print(g.bfs(2,5))

Graph of 6 edges:
0:[2, 1]
2:[3]
1:[2]
3:[]
4:[5]
5:[]
dfs:
[0, 2, 1, 3, 0, 2, 3, 1, 0, 2, 3, 1, 0, 2, 3, 1, 0]
bfs:
0 []
1 [2]
2 []
3 []
2
None
None
